# Libreria `cppyy`
Esta libreria permite integrar Python y C++ de una forma más directa.

In [1]:
import cppyy

# Ejemplo basico
cppyy.include('zlib.h')
cppyy.load_library('libz')
cppyy.gbl.zlibVersion()

'1.3.1'

## Importando GeoJSON desde python
Usando la librería `cppyy` podemos delegar todo el procesamiento JSON a Python y hacer los cálculos correspondientes en C++.

In [2]:
import json

with open('a4.json', 'r') as file:
    a4json = json.load(file)
with open('b4.json', 'r') as file:
    b4json = json.load(file)
    
a4json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'coordinates': [-99.74643387461701, 19.994418410124936],
    'type': 'Point'}},
  {'type': 'Feature',
   'properties': {},
   'geometry': {'coordinates': [-98.93927229373823, 19.30239486765392],
    'type': 'Point'}},
  {'type': 'Feature',
   'properties': {},
   'geometry': {'coordinates': [-97.72492992982917, 19.45065004363352],
    'type': 'Point'}},
  {'type': 'Feature',
   'properties': {},
   'geometry': {'coordinates': [-96.51613813793878, 19.976112338613415],
    'type': 'Point'}}]}

In [3]:
import shapely

shapely.from_geojson(json.dumps(a4json)), shapely.from_geojson(json.dumps(b4json))

(<GEOMETRYCOLLECTION (POINT (-99.746 19.994), POINT (-98.939 19.302), POINT (...>,
 <GEOMETRYCOLLECTION (POINT (-99.656 19.136), POINT (-98.73 19.794), POINT (-...>)

### Creación de clases GeoJSON en python
Esto permite mapear los GeoJSON a un objeto en python y poder después traducir esto a un objeto C++ como una alternativa, o manipular los datos de una forma más orientada a objetos.

In [4]:
class GeoJSONCollection:
    def __init__(self, features=None):
        self.type = "FeatureCollection"
        self.features = features or []

class Geometry:
    def __init__(self, coordinates=None):
        self.coordinates = coordinates

class GeoJSONFeature:
    def __init__(self, geometry_type, properties=None):
        self.geometry_type = geometry_type
        self.properties = properties or {}
        self.geometry = Geometry()

class GeoJSONPoint(GeoJSONFeature):
    def __init__(self, coordinates, properties=None):
        super().__init__("Point", properties)
          
        self.geometry.coordinates = coordinates

    @property
    def x(self):
        return self.geometry.coordinates[0]

    @property
    def y(self):
        return self.geometry.coordinates[1]

In [5]:
def load_geojson(data):
    features = []
    
    for feature in data.get("features", []):
        geometry_type = feature.get("geometry", {}).get("type")

        if geometry_type == "Point":
            properties = feature.get("properties", {})
            coordinates = feature.get("geometry", {}).get("coordinates")
            point = GeoJSONPoint(coordinates, properties)

            features.append(point)

    return GeoJSONCollection(features)

In [21]:
a4 = load_geojson(a4json)
b4 = load_geojson(b4json)
len(a4.features)

4

## Ejemplo básico con puntos
Se creo una libreria en C++ que permite crear instancias de tipo `Point` y llamar dentro de esta instancia el método `distanceTo` que calcula la distancia euclideana entre 2 puntos.

In [9]:
cppyy.include('/home/edev/devel/similarity-measures/pointlib/point.h')
cppyy.load_library('/home/edev/devel/similarity-measures/pointlib/libpoint.so')

point1 = cppyy.gbl.Point(1.0, 2.0)
point2 = cppyy.gbl.Point(3.0, 4.0)

distance = point1.distanceTo(point2)

print(f"Distance between points {distance}")

Distance between points 2.8284271247461903


Comparación con shapely:

In [13]:
p1 = shapely.geometry.point.Point(1.0, 2.0)
p2 = shapely.geometry.point.Point(3.0, 4.0)

shapely.distance(p1, p2)

2.8284271247461903

## Distancia euclideana
Se desarrolló una librería en C++ que calcula la distancia euclideana de colecciones de puntos del mismo tamaño, usando básicamente la siguiente función:

```cpp
double distance(const std::vector<double>& p,
        		const std::vector<double>& q)
{
	// Check if arrays have the same size
	if (p.size() != q.size()) {
		std::cerr << "Error: Arrays must have the same size." << std::endl;
		return -1.0; // Return an error value
	}

	double sum_of_squares = 0.0;
	for (size_t i = 0; i < p.size(); ++i) {
		double diff = p[i] - q[i];
		sum_of_squares += diff * diff;
	}

	return sqrt(sum_of_squares);
}
````

In [33]:
cppyy.include('/home/edev/devel/similarity-measures/src/euclidean.h')
cppyy.load_library('/home/edev/devel/similarity-measures/build/libeuclidean.so')
lib = cppyy.gbl

lib.distance([1.0, 2.0], [3.0, 4.0])

2.8284271247461903

De esta forma, ahora solo queda adaptar una función que trabaje con los GeoJSON importados.

In [61]:
def distance(c1: GeoJSONCollection, c2: GeoJSONCollection) -> float:
    p = [point.x for point in c1.features] + [point.y for point in c1.features]
    q = [point.x for point in c2.features] + [point.y for point in c2.features]

    return lib.distance(p, q)
    
distance(a4, b4)

1.214961596758731

### Comparación con Shapely
Los resultados son muy distintos tanto en colección de puntos como en `LineString`.

In [35]:
shapely.distance(shapely.from_geojson(json.dumps(a4json)), shapely.from_geojson(json.dumps(b4json)))

0.14962914341371103

In [56]:
sa4 = shapely.LineString([[-99.74643387461701, 19.994418410124936], [-98.93927229373823, 19.30239486765392], [-97.72492992982917, 19.45065004363352], [-96.51613813793878, 19.976112338613415]])
sb4 = shapely.LineString([[-99.65598412083149, 19.136428082255215], [-98.729859754318, 19.793728595661683], [-97.17610512705075, 19.100017568992712], [-96.65907954890238, 19.931878487430687]])

shapely.distance(sa4, sb4)

0.0

Se verificaron los resultados con Wolfram:

![distance_wolfram](wa.png)